In [40]:
from IPython.display import clear_output
from datetime import datetime, timedelta
from pathlib import Path
import pandas as pd
import holoviews as hv
from holoviews import opts
import hvplot.pandas
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import (
    QuantileTransformer,
    RobustScaler,
    OneHotEncoder,
    StandardScaler,
)
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    roc_auc_score,
    recall_score,
    roc_curve,
)
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from scipy import stats
from tqdm.notebook import tqdm
import helper_functions as hf

clear_output()

In [41]:
hv.extension("bokeh")
hvplot.extension("bokeh")
# for setting the number of columns to display in the notebook
pd.set_option("display.max_columns", 50)
clear_output()

Use helper functions to get the file

In [42]:
zip_path = Path("data/relax_challenge.zip")

# use the functions to extract the zipfile
if hf.check_zipfile(zip_path):
  target_dir = zip_path.parent / zip_path.stem
  hf.create_target_directory(target_dir)
  hf.extract_zipfile(zip_path, target_dir)

Extracted data\relax_challenge.zip to data\relax_challenge


In [43]:
user_engagement_path = (
    "./data/relax_challenge/relax_challenge/takehome_user_engagement.csv")

users_path = "data/relax_challenge/relax_challenge/takehome_users.csv"

users_engagement_df = pd.read_csv(user_engagement_path)
display(users_engagement_df.sample(3))
users_engagement_df.info()
display(users_engagement_df.describe(include="all").round(2).T.fillna(""))
# drop visited column
users_engagement_df.drop(columns="visited", inplace=True)

# convert time_stamp to datetime
users_engagement_df["time_stamp"] = pd.to_datetime(
    users_engagement_df["time_stamp"])
# # Convert user_id to string
# users_engagement_df['user_id'] = users_engagement_df['user_id'].astype(
#     'string').str.zfill(5)

,time_stamp,user_id,visited
124919,2013-04-06 01:51:47,7107,1
89985,2012-12-02 16:29:33,4954,1
40463,2013-07-07 23:14:30,2474,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207917 entries, 0 to 207916
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   time_stamp  207917 non-null  object
 1   user_id     207917 non-null  int64 
 2   visited     207917 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.8+ MB


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
time_stamp,207917.0,207220,2013-04-06 21:21:37,2,,,,,,,
user_id,207917.0,,,,5913.31,3394.94,1.0,3087.0,5682.0,8944.0,12000.0
visited,207917.0,,,,1.0,0.0,1.0,1.0,1.0,1.0,1.0


In [44]:
# show the description of the dataframe
display(users_engagement_df.describe(include="all").round(2).T.fillna(""))
# get value counts of user_id
user_id_counts = users_engagement_df["user_id"].value_counts()
print(f"number of unique users: {len(user_id_counts)}")
user_id_counts

,count,mean,min,25%,50%,75%,max,std
time_stamp,207917.0,2013-10-30 05:06:45.648763648,2012-05-31 08:20:06,2013-07-16 20:17:21,2013-12-03 06:38:34,2014-03-13 08:00:24,2014-06-06 14:58:50,
user_id,207917.0,5913.31,1.0,3087.0,5682.0,8944.0,12000.0,3394.94


number of unique users: 8823


user_id
3623     606
906      600
1811     593
7590     590
8068     585
        ... 
4699       1
4698       1
4697       1
4696       1
12000      1
Name: count, Length: 8823, dtype: int64

Only 8,823 accounts out of 12,000 logged in and were found in the engagement dataset. The other accounts we do not have any login/engagement data for.

Find the `adopted_user`s.<br>
**Criteria: Users with 3 logins in a 7-day period.**

In [45]:
# Convert the timestamp to date
users_engagement_df["date"] = pd.to_datetime(
    users_engagement_df["time_stamp"]).dt.date

# Drop duplicates based on user_id and date to get unique logins
user_engagement_datewise = users_engagement_df[[
    "user_id", "date"
]].drop_duplicates(subset=["user_id", "date"])

# Sort the DataFrame by user_id and date
user_engagement_datewise.sort_values(by=["user_id", "date"], inplace=True)

# Calculate the difference in days between the current and previous login date for each user
user_engagement_datewise["date_diff"] = user_engagement_datewise.groupby(
    "user_id")["date"].diff()
user_engagement_datewise["date_diff"] = user_engagement_datewise[
    "date_diff"].apply(lambda x: pd.Timedelta(x).days)

# Fill NA values with 0
user_engagement_datewise["date_diff"] = user_engagement_datewise[
    "date_diff"].fillna(0)

# Calculate the difference in days between the current and the login date two rows back for each user
user_engagement_datewise["date_diff_2"] = (user_engagement_datewise.groupby(
    "user_id")["date"].diff(2).apply(lambda x: pd.Timedelta(x).days))

# Filter the DataFrame to get users with 3 logins in a 7-day period
adopted_users_df = (user_engagement_datewise[
    user_engagement_datewise["date_diff_2"] < 8].groupby("user_id").first())

# Reset the index and rename the columns
adopted_users_df = adopted_users_df[["date"]].reset_index()
adopted_users_df.columns = ["user_id", "date_became_adopted"]

# Mark these users as adopted users
adopted_users_df["adopted_user"] = 1

adopted_users = adopted_users_df["user_id"].unique()
print(f"Number of adopted users: {len(adopted_users)}")

Number of adopted users: 1656


In [46]:
# get the highest date in the dataset
max_timestamp = users_engagement_df["time_stamp"].max()
print(f"The most recent user_login date is: \n{max_timestamp}")

The most recent user_login date is: 
2014-06-06 14:58:50


In [47]:
# Get a cumcount of the number of logins for each user at each row
user_engagement_datewise["login_count"] = (
    user_engagement_datewise.groupby("user_id").cumcount() + 1)
# get a cumsum of the number of days since the first login for each user at each row
user_engagement_datewise[
    "days_since_first_login"] = user_engagement_datewise.groupby(
        "user_id")["date_diff"].cumsum()
# get the avg time between logins for each user at each row
user_engagement_datewise["avg_time_between_logins"] = (
    user_engagement_datewise["days_since_first_login"] /
    user_engagement_datewise["login_count"]).round(1)

user_engagement_datewise.loc[
    user_engagement_datewise["avg_time_between_logins"] > 0]
# filter the user_engagement_datewise to get the rows of users who are adopted
user_engagement_datewise[user_engagement_datewise["user_id"].isin(
    adopted_users)]
# get the login_count number when they became an adopted user
# merge on the user_id and the date_became_adopted with user_id and date respectively
adopted_users_df = adopted_users_df.rename(
    columns={"date_became_adopted": "date"})
adoption_df = adopted_users_df.merge(user_engagement_datewise,
                                     on=["user_id", "date"])
# get a cumulative count of the adopted users in chronological order
adoption_df["date"] = pd.to_datetime(adoption_df["date"])
adoption_df.sort_values("date", inplace=True)
adoption_df["cum_adopted"] = adoption_df["adopted_user"].cumsum()
display(adoption_df.head())
# plot the cumulative adopted users
adoption_df.sort_values(by=["date"]).hvplot.scatter(
    x="date", y="cum_adopted", size=5,
    title="Cumulative Adopted Users").opts(active_tools=["box_zoom"])

,user_id,date,adopted_user,date_diff,date_diff_2,login_count,days_since_first_login,avg_time_between_logins,cum_adopted
224,1693,2012-06-10,1,1.0,5.0,4,10.0,2.5,1
106,728,2012-06-16,1,1.0,5.0,3,5.0,1.7,2
206,1525,2012-06-16,1,5.0,7.0,3,7.0,2.3,3
1624,11764,2012-06-17,1,2.0,4.0,3,4.0,1.3,4
1020,7590,2012-06-18,1,5.0,7.0,3,7.0,2.3,5


:Scatter   [date]   (cum_adopted)

In [48]:
# get the count of the adopted users for each month
adoption_df["month"] = adoption_df["date"].dt.to_period("M")
adoption_df["month_adoption_count"] = adoption_df.groupby(
    "month")["adopted_user"].transform("sum")
# check the adoptions for each month and plot on a bar chart
monthly_adoption_count = (adoption_df[["month", "month_adoption_count"
                                       ]].drop_duplicates().set_index("month"))
monthly_adoption_count.hvplot() * monthly_adoption_count.hvplot.scatter(
    height=600, title="Adoptions per Month").opts(active_tools=["box_zoom"])

:Overlay
   .Curve.I   :Curve   [month]   (month_adoption_count)
   .Scatter.I :Scatter   [month]   (month_adoption_count)

In [49]:
# get the count of the adopted users for each week
adoption_df["week"] = adoption_df["date"].dt.to_period("W")
adoption_df["week_adoption_count"] = adoption_df.groupby(
    "week")["adopted_user"].transform("sum")

# check the weekly adoptions
weekly_adoption_count = (adoption_df[["date", "adopted_user"
                                      ]].set_index("date").resample("W").sum())
weekly_adoption_count.hvplot() * weekly_adoption_count.hvplot(
    height=600, title="Adoptions per Week").opts(active_tools=["box_zoom"])
# smooth out the weekly adoptions
(weekly_adoption_count.rolling(window=7, center=True,
                               min_periods=1).mean().hvplot() *
 weekly_adoption_count.hvplot.scatter(
     title="Adoptions per Week", color="gray")).opts(active_tools=["box_zoom"],
                                                     height=500,
                                                     width=1000,
                                                     show_grid=True,
                                                     xlabel="")
# adoption_df

:Overlay
   .Curve.I   :Curve   [date]   (adopted_user)
   .Scatter.I :Scatter   [date]   (adopted_user)

In [50]:
weekly_adoption_count

,adopted_user
date,
2012-06-10,1
2012-06-17,3
2012-06-24,5
2012-07-01,3
2012-07-08,11
...,...
2014-05-11,27
2014-05-18,26
2014-05-25,19


In [51]:
# get the count of the adopted users for each day
adoption_df["day"] = adoption_df["date"].dt.to_period("D")
adoption_df["day_adoption_count"] = adoption_df.groupby(
    "day")["adopted_user"].transform("sum")
# check the daily adoptions
daily_adoption_count = (adoption_df[["date", "adopted_user"
                                     ]].set_index("date").resample("D").sum())
daily_adoption_count.hvplot(grid=True) * daily_adoption_count.hvplot.scatter(
    height=600, title="Adoptions per Day").opts(active_tools=["box_zoom"])

:Overlay
   .Curve.I   :Curve   [date]   (adopted_user)
   .Scatter.I :Scatter   [date]   (adopted_user)

Small number of adopted users, 13% of the users are adopted. The dataset is imbalanced.

In [52]:
users_df = pd.read_csv(users_path, encoding="latin-1")
users_df.info()
display(users_df.sample(3))
print(f"Statistics of the users dataframe")
users_df.describe(include="all").round(2).T.sort_values(by="unique").fillna("")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   object_id                   12000 non-null  int64  
 1   creation_time               12000 non-null  object 
 2   name                        12000 non-null  object 
 3   email                       12000 non-null  object 
 4   creation_source             12000 non-null  object 
 5   last_session_creation_time  8823 non-null   float64
 6   opted_in_to_mailing_list    12000 non-null  int64  
 7   enabled_for_marketing_drip  12000 non-null  int64  
 8   org_id                      12000 non-null  int64  
 9   invited_by_user_id          6417 non-null   float64
dtypes: float64(2), int64(4), object(4)
memory usage: 937.6+ KB


,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
10773,10774,2012-09-30 04:51:15,Olesen Natasja,NatasjaTOlesen@gustr.com,ORG_INVITE,1.349067e+09,1,1,94,1525.0
7042,7043,2012-10-27 05:40:43,Fernandes Cauã,CauaOliveiraFernandes@gmail.com,ORG_INVITE,1.351316e+09,1,0,344,651.0
4002,4003,2014-04-08 23:36:49,Henderson Justin,JustinHenderson@gustr.com,GUEST_INVITE,1.400888e+09,0,1,79,10119.0


Statistics of the users dataframe


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
creation_source,12000.0,5,ORG_INVITE,4254,,,,,,,
name,12000.0,11355,Araujo Gabriela,5,,,,,,,
email,12000.0,11980,AlfieLane@yahoo.com,2,,,,,,,
creation_time,12000.0,11996,2014-02-11 17:57:53,2,,,,,,,
object_id,12000.0,,,,6000.5,3464.25,1.0,3000.75,6000.5,9000.25,12000.0
last_session_creation_time,8823.0,,,,1379279305.7,19531160.79,1338452406.0,1363194965.0,1382888470.0,1398442604.0,1402066730.0
opted_in_to_mailing_list,12000.0,,,,0.25,0.43,0.0,0.0,0.0,0.0,1.0
enabled_for_marketing_drip,12000.0,,,,0.15,0.36,0.0,0.0,0.0,0.0,1.0
org_id,12000.0,,,,141.88,124.06,0.0,29.0,108.0,238.25,416.0
invited_by_user_id,6417.0,,,,5962.96,3383.76,3.0,3058.0,5954.0,8817.0,11999.0


In [53]:
users_df["last_session_creation_time"] = pd.to_datetime(
    users_df["last_session_creation_time"], unit="s")
users_df["creation_time"] = pd.to_datetime(users_df["creation_time"])
display(users_df.sample(3))
# describe the 2 time columns
users_df[["creation_time",
          "last_session_creation_time"]].describe(include="all").T.fillna("")

# Calculate the recency of the user's last session
users_df["recency"] = (max_timestamp -
                       users_df["last_session_creation_time"]).dt.days
# Describe the recency column
users_df.recency.describe()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
9206,9207,2014-01-03 15:18:53,Mahmood Isabelle,IsabelleMahmood@gmail.com,SIGNUP_GOOGLE_AUTH,2014-01-12 15:18:53,0,0,192,NaN
2372,2373,2013-11-22 19:44:45,Kuster Sven,SvenKuster@yahoo.com,SIGNUP,2013-11-22 19:44:45,0,0,68,NaN
5881,5882,2014-05-19 06:22:14,Farber Martina,MartinaFarber@gustr.com,ORG_INVITE,2014-06-04 06:22:14,0,0,83,4096.0


count    8823.000000
mean      263.244361
std       226.056408
min         0.000000
25%        41.000000
50%       221.000000
75%       449.000000
max       736.000000
Name: recency, dtype: float64

In [54]:
# Create a histogram of the recency column
users_df["recency"].hvplot.hist(title="Recency of Last Session",
                                bins=20,
                                color="orange").opts(active_tools=["box_zoom"],
                                                     height=300,
                                                     width=600)

:Histogram   [recency]   (recency_count)

In [55]:
# examine the missing values for the last_session_creation_time
display(users_df[users_df["last_session_creation_time"].isna()].describe(
    include="all").T.fillna(""))
# see if the object_id of the null dataframe is in the engagement data as the user_id
null_df = users_df[users_df["last_session_creation_time"].isna()]
null_df["object_id"].isin(users_engagement_df["user_id"]).sum()
print(
    f"Number of users with missing last_session_creation_time: {len(null_df):,}"
)

,count,unique,top,freq,mean,min,25%,50%,75%,max,std
object_id,3177.0,,,,5946.84073,8.0,2919.0,5941.0,8909.0,11994.0,3464.211901
creation_time,3177.0,,,,2013-07-10 17:30:15.857412608,2012-05-31 18:12:49,2013-01-09 05:27:00,2013-07-21 05:44:10,2014-01-23 05:34:03,2014-05-30 22:34:31,
name,3177.0,3133,Rodrigues Ryan,3,,,,,,,
email,3177.0,3176,ThomasBrandt@gmail.com,2,,,,,,,
creation_source,3177.0,4,PERSONAL_PROJECTS,1347,,,,,,,
last_session_creation_time,0.0,,,,,,,,,,
opted_in_to_mailing_list,3177.0,,,,0.241737,0.0,0.0,0.0,0.0,1.0,0.428203
enabled_for_marketing_drip,3177.0,,,,0.141958,0.0,0.0,0.0,0.0,1.0,0.349062
org_id,3177.0,,,,139.974819,0.0,28.0,104.0,237.0,415.0,123.723159
invited_by_user_id,1641.0,,,,5911.913467,7.0,2998.0,5978.0,8664.0,11999.0,3353.670938


Number of users with missing last_session_creation_time: 3,177


In [56]:
# Missing value data for org_id
users_df.loc[users_df["last_session_creation_time"].isna(
)]["org_id"].value_counts().sort_index().hvplot(
    title="Org ID of rows with missing last_session_creation_time",
    color="silver").opts(active_tools=["box_zoom"], height=300, width=600)

:Curve   [org_id]   (count)

We found that `3177` users were missing data from the `last_session_creation_time` column. These users were also missing data from the `user_engagement` dataset. We therefore could not determine if these users were adopted users or not. Although this was a sizeable portion of the dataset, we decided to drop these users from the dataset.

In [57]:
# non null data
users_df[~users_df["last_session_creation_time"].isna(
)]["org_id"].value_counts().sort_index().hvplot()

non_null_df = users_df[~users_df["last_session_creation_time"].isna()]
non_null_df['org_id'].value_counts().sort_index().hvplot(
    title="Org ID of rows with non-missing last_session_creation_time",
    color="orange").opts(active_tools=["box_zoom"], height=300, width=600)


:Curve   [org_id]   (count)

In [58]:
# drop the rows with the missing last_session_creation_time
users_df = users_df[~users_df["last_session_creation_time"].isna()]
users_df.info()
# look at the missing values
users_df.isna().sum()
print(
    f"\nNumber of missing values in invited_by_user_id: \n{users_df['invited_by_user_id'].isna().sum()}"
)

# investigate the missing values in invited_by_user_id
users_df[users_df["invited_by_user_id"].isna()].head(3)

<class 'pandas.core.frame.DataFrame'>
Index: 8823 entries, 0 to 11999
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   object_id                   8823 non-null   int64         
 1   creation_time               8823 non-null   datetime64[ns]
 2   name                        8823 non-null   object        
 3   email                       8823 non-null   object        
 4   creation_source             8823 non-null   object        
 5   last_session_creation_time  8823 non-null   datetime64[ns]
 6   opted_in_to_mailing_list    8823 non-null   int64         
 7   enabled_for_marketing_drip  8823 non-null   int64         
 8   org_id                      8823 non-null   int64         
 9   invited_by_user_id          4776 non-null   float64       
 10  recency                     8823 non-null   float64       
dtypes: datetime64[ns](2), float64(2), int64(4), object(3)
memory

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,recency
6,7,2012-12-16 13:24:32,Sewell Tyler,TylerSewell@jourrapide.com,SIGNUP,2012-12-20 13:24:32,0,1,37,NaN,533.0
10,11,2013-12-26 03:55:54,Paulsen Malthe,MaltheAPaulsen@gustr.com,SIGNUP,2013-12-27 03:55:54,0,0,69,NaN,161.0
13,14,2012-10-11 16:14:33,Rivera Bret,BretKRivera@gmail.com,SIGNUP,2012-10-12 16:14:33,0,0,0,NaN,601.0


In [59]:
null_df = users_df[users_df["invited_by_user_id"].isna()]
# Check if the object_id of the null dataframe is in the engagement data as the user_id
null_df["object_id"].isin(users_engagement_df["user_id"]).sum()

4047

All these rows with null values are IN the `engagement_dataset`. We can use these rows. The missing values is not a problem . They are because noone invited these users as they are not from an `ORG_INVITE`  nor a `GUEST_INVITE`. We can fill these missing values with `00000` as that is not a `user_id`.

In [60]:
# Look at the count of the creation_source and the org_id
display(null_df["creation_source"].value_counts())
# Look at the invited_by_user_id for the null dataframe
users_df["invited_by_user_id"] = users_df["invited_by_user_id"].fillna("00000")
# look at the info and description of the dataframe
users_df.info()
users_df.describe(include="all").T.sort_values(by="unique").fillna("")

creation_source
SIGNUP                1898
SIGNUP_GOOGLE_AUTH    1385
PERSONAL_PROJECTS      764
Name: count, dtype: int64

<class 'pandas.core.frame.DataFrame'>
Index: 8823 entries, 0 to 11999
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   object_id                   8823 non-null   int64         
 1   creation_time               8823 non-null   datetime64[ns]
 2   name                        8823 non-null   object        
 3   email                       8823 non-null   object        
 4   creation_source             8823 non-null   object        
 5   last_session_creation_time  8823 non-null   datetime64[ns]
 6   opted_in_to_mailing_list    8823 non-null   int64         
 7   enabled_for_marketing_drip  8823 non-null   int64         
 8   org_id                      8823 non-null   int64         
 9   invited_by_user_id          8823 non-null   object        
 10  recency                     8823 non-null   float64       
dtypes: datetime64[ns](2), float64(1), int64(4), object(4)
memory

,count,unique,top,freq,mean,min,25%,50%,75%,max,std
creation_source,8823.0,5,ORG_INVITE,3188,,,,,,,
invited_by_user_id,8823.0,2230,00000,4047,,,,,,,
name,8823.0,8453,Correia Leonardo,4,,,,,,,
email,8823.0,8810,MarkoSeiler@yahoo.com,2,,,,,,,
object_id,8823.0,,,,6019.821716,1.0,3017.5,6034.0,9029.5,12000.0,3464.251001
creation_time,8823.0,,,,2013-07-18 15:48:32.228833792,2012-05-31 00:43:27,2013-01-18 22:39:45.500000,2013-08-09 22:08:11,2014-01-30 00:21:54,2014-05-30 23:59:19,
last_session_creation_time,8823.0,,,,2013-09-15 21:08:25.700441856,2012-05-31 08:20:06,2013-03-13 17:16:05,2013-10-27 15:41:10,2014-04-25 16:16:44,2014-06-06 14:58:50,
opted_in_to_mailing_list,8823.0,,,,0.252295,0.0,0.0,0.0,1.0,1.0,0.434354
enabled_for_marketing_drip,8823.0,,,,0.151989,0.0,0.0,0.0,0.0,1.0,0.359031
org_id,8823.0,,,,142.572254,0.0,30.0,109.0,239.0,416.0,124.176422


In [61]:
# do a KMeans clustering on the recency column
# use a range of clusters from 2 to 10 and plot the inertia
inertia = {}
for k in range(1, 11):
  kmeans = KMeans(n_clusters=k, random_state=628)
  kmeans.fit(users_df[["recency"]])
  inertia[k] = kmeans.inertia_

# plot the inertia
pd.Series(inertia).hvplot.line(
    title="Inertia of KMeans Clustering",
    xlabel="Number of Clusters",
    xlim=(0, 11),
    ylabel="Inertia",
).opts(xticks=list(range(11)), active_tools=["box_zoom"])

:Curve   [index]   (0)

We can go with 3 clusters to see if we can find any patterns in the data.

In [62]:
# rename the object_id column to user_id
users_df = users_df.rename(columns={"object_id": "user_id"})

In [63]:
# build 3 clusters for the recency column
k = 3
k_colors = {0: "silver", 1: "orange", 2: "green", 3: "purple"}
kmeans = KMeans(n_clusters=k, random_state=628)
users_df["recency_cluster"] = kmeans.fit_predict(users_df[["recency"]])

In [64]:
# Get the count of the recency clusters
users_df["recency_cluster"].value_counts()
# get the recency statistics for each cluster
users_df.groupby("recency_cluster")["recency"].describe().T

recency_cluster,0,1,2
count,2229.000000,4008.000000,2586.000000
mean,582.954688,54.623004,311.009281
std,82.763513,54.895730,76.945246
min,447.000000,0.000000,183.000000
25%,511.000000,8.000000,244.000000
50%,581.000000,30.000000,307.500000
75%,652.000000,96.000000,377.000000
max,736.000000,182.000000,446.000000


In [65]:
# Calculate mean recency for each cluster
cluster_recency = (
    users_df.groupby("recency_cluster")["recency"].mean().sort_values(
        ascending=False))

# Create a dictionary mapping the old cluster names to the new ones
cluster_mapping = {
    cluster: i
    for i, cluster in enumerate(cluster_recency.index)
}

# Use the map function to replace the old cluster names with the new ones
users_df["recency_cluster"] = users_df["recency_cluster"].map(cluster_mapping)
users_df["recency_cluster"].value_counts()

recency_cluster
2    4008
1    2586
0    2229
Name: count, dtype: int64

In [67]:
rfm_opts = dict(
    active_tools=["box_zoom"],
    height=300,
    width=600,
    xlabel="",
)

recency_plot = users_df.hvplot.scatter(
    x="user_id",
    y="recency",
    c="recency_cluster",
    cmap=k_colors,
    title="Recency Clusters",
).opts(**rfm_opts)
recency_plot

:Scatter   [user_id]   (recency,recency_cluster)

In [68]:
# get the frequency of the users and add to the users dataframe
users_df = users_df.merge(
    users_engagement_df["user_id"].value_counts().reset_index(
        name="frequency"),
    how="left",
)

# plot the frequency of the users
users_df["frequency"].hvplot.hist(
    title="Frequency of User Logins",
    bins=20,
    color="orange",
).opts(active_tools=["box_zoom"], height=300, width=600)

:Histogram   [frequency]   (frequency_count)

Apply the same logic for having frequency clusters.


In [69]:
kmeans = KMeans(n_clusters=k, random_state=628)
users_df["frequency_cluster"] = kmeans.fit_predict(users_df[["frequency"]])

In [70]:
# Statistics of the frequency clusters
users_df.groupby("frequency_cluster")["frequency"].describe().T

frequency_cluster,0,1,2
count,8064.000000,272.000000,487.000000
mean,4.123636,374.261029,149.620123
std,10.303973,83.379019,52.012477
min,1.000000,263.000000,77.000000
25%,1.000000,304.000000,105.000000
50%,1.000000,355.500000,141.000000
75%,1.000000,434.250000,191.000000
max,76.000000,606.000000,261.000000


In [71]:
# Calculate the mean frequency of each cluster and sort the values ascending True this time
cluster_frequency = (
    users_df.groupby("frequency_cluster")["frequency"].mean().sort_values())
# Create a dictionary mapping the old cluster names to the new ones
cluster_mapping = {
    cluster: i
    for i, cluster in enumerate(cluster_frequency.index)
}
# Use the map function to replace the old cluster names with the new ones
users_df["frequency_cluster"] = users_df["frequency_cluster"].map(
    cluster_mapping)
users_df["frequency_cluster"].value_counts()

frequency_cluster
0    8064
1     487
2     272
Name: count, dtype: int64

In [72]:
frequency_plot = users_df.hvplot.scatter(
    x="user_id",
    y="frequency",
    c="frequency_cluster",
    cmap=k_colors,
    title="Frequency Clusters",
).opts(**rfm_opts)
frequency_plot

:Scatter   [user_id]   (frequency,frequency_cluster)

Since we do not have any monetary data, we can use the `num_referrals` for this feature. We can get a value for this counting the `user_id` in the `invited_by_user_id` column.

In [73]:
# get the count of the user_id in the 'invited_by_user_id' column
num_referrals = users_df["invited_by_user_id"].astype(
    "int").value_counts().sort_index()
# map the number of referrals to the user_id
users_df["num_referrals"] = users_df["user_id"].map(num_referrals).fillna(
    0).astype(int)
# plot the number of referrals
users_df["num_referrals"].hvplot.hist(
    title="Number of Referrals",
    bins=10,
    color="orange",
).opts(active_tools=["box_zoom"], height=300, width=600)

:Histogram   [num_referrals]   (num_referrals_count)

In [74]:
# Get the clusters for the number of referrals
kmeans = KMeans(n_clusters=k, random_state=628)
users_df["referral_cluster"] = kmeans.fit_predict(users_df[["num_referrals"]])

In [75]:
# Get the statistics of the referral clusters
users_df.groupby("referral_cluster")["num_referrals"].describe().T

referral_cluster,0,1,2
count,1358.000000,7189.0,276.000000
mean,1.598675,0.0,5.043478
std,0.749491,0.0,1.177792
min,1.000000,0.0,4.000000
25%,1.000000,0.0,4.000000
50%,1.000000,0.0,5.000000
75%,2.000000,0.0,6.000000
max,3.000000,0.0,10.000000


In [76]:
# Sort by the mean number of referrals for each cluster and ascending like the frequency clusters
cluster_referral = (
    users_df.groupby("referral_cluster")["num_referrals"].mean().sort_values())
# Create a dictionary mapping the old cluster names to the new ones
cluster_mapping = {
    cluster: i
    for i, cluster in enumerate(cluster_referral.index)
}
# Use the map function to replace the old cluster names with the new ones
users_df["referral_cluster"] = users_df["referral_cluster"].map(
    cluster_mapping)
users_df["referral_cluster"].value_counts()

referral_cluster
0    7189
1    1358
2     276
Name: count, dtype: int64

In [77]:
# show corrected scatter plot
referral_plot = users_df.hvplot.scatter(
    x="user_id",
    y="num_referrals",
    c="referral_cluster",
    cmap=k_colors,
    title="Recency vs Frequency",
).opts(**rfm_opts)

# Layout with the plots
layout = recency_plot + frequency_plot + referral_plot
layout.cols(1)

:Layout
   .Scatter.I   :Scatter   [user_id]   (recency,recency_cluster)
   .Scatter.II  :Scatter   [user_id]   (frequency,frequency_cluster)
   .Scatter.III :Scatter   [user_id]   (num_referrals,referral_cluster)

In [98]:
rfm_df = users_df[[
    "user_id", "recency_cluster", "frequency_cluster", "referral_cluster",
    'recency', 'frequency', 'num_referrals'
]]
rfm_df = rfm_df.set_index("user_id")
# change the clusters to numerical values
rfm_df["score"] = rfm_df["recency_cluster"] + rfm_df[
    "frequency_cluster"] + rfm_df["referral_cluster"]
rfm_df["score"].value_counts().sort_index().hvplot.bar(
    title="RFM Score Distribution",
    xlabel="RFM Score",
    ylabel="Count",
    color="orange",
).opts(active_tools=["box_zoom"], height=300, width=600)

:Bars   [score]   (count)

In [95]:
adopted_users_df

,user_id,date,adopted_user
0,2,2014-02-09,1
1,10,2013-02-06,1
2,20,2014-03-13,1
3,33,2014-03-23,1
4,42,2012-12-25,1
...,...,...,...
1651,11965,2014-05-02,1
1652,11967,2014-03-10,1
1653,11969,2013-06-03,1
1654,11975,2013-05-29,1


In [100]:
rfm_adopted_df = rfm_df.reset_index().merge(
    adopted_users_df, how='left').drop(columns=['date']).fillna(0)

rfm_adopted_df.hvplot.scatter(x='score',
                              y='recency',
                              c='adopted_user',
                              cmap='viridis',
                              title='Recency vs Frequency',
                              height=500,
                              width=800,
                              hover_cols='all')

:Scatter   [score]   (recency,adopted_user,index,user_id,recency_cluster,frequency_cluster,referral_cluster,frequency,num_referrals)

In [232]:
all_users_df = users_df.merge(adopted_users_df, how="left")
all_users_df

,user_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,recency,recency_cluster,frequency,frequency_cluster,num_referrals,referral_cluster,date,adopted_user
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,2014-04-22 03:53:30,1,0,11,10803.0,45.0,2,1,0,0,0,NaN,NaN
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,2014-03-31 03:45:04,0,0,1,316.0,67.0,2,14,0,0,0,2014-02-09,1.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,2013-03-19 23:14:52,0,0,94,1525.0,443.0,1,1,0,1,1,NaN,NaN
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,2013-05-22 08:09:28,0,0,1,5151.0,380.0,1,1,0,0,0,NaN,NaN
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,2013-01-22 10:14:20,0,0,193,5240.0,500.0,0,1,0,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8818,11996,2013-09-06 06:14:15,Meier Sophia,SophiaMeier@gustr.com,ORG_INVITE,2013-09-06 06:14:15,0,0,89,8263.0,273.0,1,1,0,0,0,NaN,NaN
8819,11997,2013-01-10 18:28:37,Fisher Amelie,AmelieFisher@gmail.com,SIGNUP_GOOGLE_AUTH,2013-01-15 18:28:37,0,0,200,00000,506.0,0,1,0,1,1,NaN,NaN
8820,11998,2014-04-27 12:45:16,Haynes Jake,JakeHaynes@cuvox.de,GUEST_INVITE,2014-04-27 12:45:16,1,1,83,8074.0,40.0,2,1,0,0,0,NaN,NaN
8821,11999,2012-05-31 11:55:59,Faber Annett,mhaerzxp@iuxiw.com,PERSONAL_PROJECTS,2012-06-02 11:55:59,0,0,6,00000,734.0,0,1,0,5,2,NaN,NaN


In [233]:
all_users_df["time_to_become_adopted"] = (
    pd.to_datetime(all_users_df["date"]) - pd.to_datetime(all_users_df["creation_time"])
).dt.days
all_users_df

,user_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,recency,recency_cluster,frequency,frequency_cluster,num_referrals,referral_cluster,date,adopted_user,time_to_become_adopted
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,2014-04-22 03:53:30,1,0,11,10803.0,45.0,2,1,0,0,0,NaN,NaN,NaN
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,2014-03-31 03:45:04,0,0,1,316.0,67.0,2,14,0,0,0,2014-02-09,1.0,85.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,2013-03-19 23:14:52,0,0,94,1525.0,443.0,1,1,0,1,1,NaN,NaN,NaN
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,2013-05-22 08:09:28,0,0,1,5151.0,380.0,1,1,0,0,0,NaN,NaN,NaN
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,2013-01-22 10:14:20,0,0,193,5240.0,500.0,0,1,0,0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8818,11996,2013-09-06 06:14:15,Meier Sophia,SophiaMeier@gustr.com,ORG_INVITE,2013-09-06 06:14:15,0,0,89,8263.0,273.0,1,1,0,0,0,NaN,NaN,NaN
8819,11997,2013-01-10 18:28:37,Fisher Amelie,AmelieFisher@gmail.com,SIGNUP_GOOGLE_AUTH,2013-01-15 18:28:37,0,0,200,00000,506.0,0,1,0,1,1,NaN,NaN,NaN
8820,11998,2014-04-27 12:45:16,Haynes Jake,JakeHaynes@cuvox.de,GUEST_INVITE,2014-04-27 12:45:16,1,1,83,8074.0,40.0,2,1,0,0,0,NaN,NaN,NaN
8821,11999,2012-05-31 11:55:59,Faber Annett,mhaerzxp@iuxiw.com,PERSONAL_PROJECTS,2012-06-02 11:55:59,0,0,6,00000,734.0,0,1,0,5,2,NaN,NaN,NaN


In [234]:
adoption_df_slim = adoption_df[
    [
        "user_id",
        "date",
        "adopted_user",
        "avg_time_between_logins",
        "login_count",
        "days_since_first_login",
    ]
]

adoption_df_slim.columns = [
    "user_id",
    "date",
    "adopted_user",
    "avg_time_bet_logins_at_adopt",
    "login_count_at_adopt",
    "account_age_at_adopt",
]

adoption_df_slim["date"] = adoption_df_slim["date"].dt.date

C:\Users\jonnr\AppData\Local\Temp\ipykernel_6688\2293998457.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adoption_df_slim["date"] = adoption_df_slim["date"].dt.date


In [235]:
all_users_df = all_users_df.merge(adoption_df_slim, how="left")

In [236]:
all_users_df["time_to_become_adopted"].hvplot.hist(
    title="Time to become adopted (days)", bins=12, xlabel=""
).opts(active_tools=["box_zoom"])

:Histogram   [time_to_become_adopted]   (time_to_become_adopted_count)

In [237]:
all_users_df.login_count_at_adopt.hvplot.hist(
    title="Login Count at Adoption", xlabel="").opts(active_tools=["box_zoom"])

:Histogram   [login_count_at_adopt]   (login_count_at_adopt_count)

In [238]:
# Fill missing values in 'adopted_user' with 0 and convert the column to integer
all_users_df["adopted_user"] = all_users_df["adopted_user"].fillna(0).astype(int)

# Fill missing values in 'logins' with 0 and convert the column to integer
all_users_df["frequency"] = all_users_df["frequency"].fillna(0).astype(int)

# Convert 'last_session_creation_time' to datetime using seconds as the unit
all_users_df["last_session_creation_time"] = pd.to_datetime(
    all_users_df["last_session_creation_time"], unit="s"
)
# get the time since last login
all_users_df["days_since_last_login"] = (
    max_timestamp - all_users_df["last_session_creation_time"]
).dt.days
all_users_df["last_session_creation_time"].hvplot.hist(
    bins=52, title="Last session creation time", xlabel=""
).opts(active_tools=["box_zoom"])

:Histogram   [last_session_creation_time]   (last_session_creation_time_count)

In [239]:
all_users_df.hvplot.hist(
    "days_since_last_login",
    by="adopted_user",
    alpha=0.5,
    title="Time since last login",
).opts(active_tools=["box_zoom"])

:NdOverlay   [adopted_user]
   :Histogram   [days_since_last_login]   (days_since_last_login_count)

In [240]:
# Convert 'creation_time' to datetime and extract the date
all_users_df["start_date"] = pd.to_datetime(all_users_df["creation_time"]).dt.date


all_users_df.hvplot.hist("start_date", alpha=0.5, title="Start Date Distribution").opts(
    active_tools=["box_zoom"]
)

:Histogram   [start_date]   (start_date_count)

In [241]:
# Convert 'last_session_creation_time' and 'creation_time' to datetime
all_users_df["last_session_creation_time"] = pd.to_datetime(
    all_users_df["last_session_creation_time"]
)
all_users_df["creation_time"] = pd.to_datetime(all_users_df["creation_time"])
# Calculate 'account_age' in days
all_users_df["account_age"] = (max_timestamp - all_users_df["creation_time"]).dt.days
# Calculate 'account_age_last_login' in days
all_users_df["account_age_last_login"] = (
    all_users_df["last_session_creation_time"] - all_users_df["creation_time"]
).dt.days
# Calculate quietness period as avg time between logins
all_users_df["time_bet_logins"] = (
    all_users_df["account_age"] / all_users_df["frequency"]
).round(1)

all_users_df["account_age_last_login"].describe()

all_users_df.sample(3)

,user_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,recency,recency_cluster,frequency,frequency_cluster,num_referrals,referral_cluster,date,adopted_user,time_to_become_adopted,avg_time_bet_logins_at_adopt,login_count_at_adopt,account_age_at_adopt,days_since_last_login,start_date,account_age,account_age_last_login,time_bet_logins
1712,2361,2014-02-14 16:23:35,Brabyn Sienna,SiennaBrabyn@hotmail.com,ORG_INVITE,2014-06-04 16:23:35,1,0,52,7287.0,1.0,2,28,0,0,0,2014-04-12,1,56.0,8.1,7.0,57.0,1,2014-02-14,111,110,4.0
1918,2634,2014-03-21 16:49:37,Hoddle Zara,ZaraHoddle@gustr.com,GUEST_INVITE,2014-03-23 16:49:37,0,0,35,11358.0,74.0,2,1,0,0,0,NaN,0,NaN,NaN,NaN,NaN,74,2014-03-21,76,2,76.0
5668,7776,2013-02-19 08:19:29,Villarreal Veronica,VeronicaSVillarreal@gustr.com,SIGNUP,2013-02-19 08:19:29,1,1,13,00000,472.0,0,1,0,0,0,NaN,0,NaN,NaN,NaN,NaN,472,2013-02-19,472,0,472.0


In [242]:
# Extract the month from 'creation_time'
all_users_df["creation_month"] = all_users_df["creation_time"].dt.month


# Convert 'user_id' to string and add leading zeros
all_users_df["user_id"] = all_users_df["user_id"].astype("string").str.zfill(5)

# Extract the domain from the 'email' column
all_users_df["domain"] = (
    all_users_df["email"].str.split("@").str[1].str.split(".").str[0]
)

# Fill missing values in 'invited_by_user_id' with 0, convert to string and add leading zeros
all_users_df["invited_by_user_id"] = (
    all_users_df["invited_by_user_id"]
    # .fillna(0)
    .astype(int)
    .astype("string")
    .str.zfill(5)
)


# Replace '00000' in 'invited_by_user_id' with 'Not Invited'
all_users_df["invited_by_user_id"] = all_users_df["invited_by_user_id"].replace(
    "00000", "Not Invited"
)

# Create a DataFrame of adopted users
adopted_users = all_users_df[all_users_df["adopted_user"] == 1]["user_id"]

# Count the number of users each user has referred
referred_df = all_users_df["invited_by_user_id"].value_counts().reset_index().iloc[1:]
referred_df.columns = ["user_id", "num_referrals"]

# Merge 'referred_df' with 'all_users_df'
all_users_df = all_users_df.merge(referred_df, how="left")

# Fill missing values in 'num_referrals' with 0
all_users_df["num_referrals"] = all_users_df["num_referrals"].fillna(0).astype(int)

# Create a new column 'was_invited' that indicates whether a user was invited or not
all_users_df["was_invited"] = all_users_df["invited_by_user_id"].apply(
    lambda x: "Invited" if x != "Not Invited" else "Not Invited"
)

In [243]:
all_users_df[(all_users_df["num_referrals"] > 0) & (all_users_df["adopted_user"] == 1)][
    [
        "user_id",
        "num_referrals",
        "adopted_user",
        "was_invited",
        "invited_by_user_id",
    ]
].sort_values(by="num_referrals", ascending=False).describe(
    include="all"
).T.sort_values(
    "unique"
).fillna(
    ""
)

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
was_invited,382.0,2,Invited,235,,,,,,,
invited_by_user_id,382.0,229,Not Invited,147,,,,,,,
user_id,382.0,382,04612,1,,,,,,,
num_referrals,382.0,,,,2.232984,1.590873,1.0,1.0,2.0,3.0,10.0
adopted_user,382.0,,,,1.0,0.0,1.0,1.0,1.0,1.0,1.0


In [244]:
all_users_df["sent_referrals"] = np.where(all_users_df["num_referrals"] > 0,
                                          "sent_referrals",
                                          "no_referrals_sent")

In [245]:
# create a column for the users who invited themselves
all_users_df["invited_self"] = (
    all_users_df["user_id"] == all_users_df["invited_by_user_id"])
all_users_df["invited_self"] = all_users_df["invited_self"].astype(int)

In [246]:
all_users_df["domain"] = all_users_df["domain"].apply(
    lambda x: (
        "other"
        if x not in ["gmail", "yahoo", "jourrapide", "cuvox", "gustr", "hotmail"]
        else x
    )
)

In [247]:
all_users_df.describe(include="all").T.sort_values("unique").fillna("")

,count,unique,top,freq,mean,min,25%,50%,75%,max,std
was_invited,8823.0,2,Invited,4776,,,,,,,
sent_referrals,8823.0,2,no_referrals_sent,7189,,,,,,,
creation_source,8823.0,5,ORG_INVITE,3188,,,,,,,
domain,8823.0,7,gmail,2930,,,,,,,
date,1656.0,625,2012-10-14,9,,,,,,,
start_date,8823.0,730,2014-05-30,53,,,,,,,
invited_by_user_id,8823.0,2230,Not Invited,4047,,,,,,,
name,8823.0,8453,Correia Leonardo,4,,,,,,,
email,8823.0,8810,MarkoSeiler@yahoo.com,2,,,,,,,
user_id,8823.0,8823,00001,1,,,,,,,


In [248]:
all_users_df.sample().T

,5266
user_id,07194
creation_time,2012-11-09 20:05:38
name,Sambell Mason
email,MasonSambell@gmail.com
creation_source,SIGNUP
last_session_creation_time,2014-05-22 20:05:38
opted_in_to_mailing_list,1
enabled_for_marketing_drip,0
org_id,50
invited_by_user_id,Not Invited


Target Variable

In [249]:
all_users_df["adopted_user"].value_counts(normalize=True).reset_index(
    name="adopted_users")

,adopted_user,adopted_users
0,0,0.812309
1,1,0.187691


In [250]:
def remove_outer_percentile_outliers(df, col, lower=0.01, upper=0.99):
    """Remove the outer percentile outliers from a column in a DataFrame."""
    lower_bound = df[col].quantile(lower)
    upper_bound = df[col].quantile(upper)
    return df[(df[col] > lower_bound) & (df[col] < upper_bound)]

In [251]:
def plot_cat_active_stacked_bars(df, cat_cols, target_col="adopted_user"):
    """Create a bar plot for each categorical column showing the proportion of active users."""

    plot_opts = dict(height=300, width=400, active_tools=["box_zoom"])
    cat_plots = []
    for col in cat_cols:
        proportions = df.groupby(col)[target_col].value_counts(normalize=True).unstack()
        plot = proportions.hvplot.barh(
            title=(f"{col}".replace("_", " ").title() if "_" in col else col.title()),
            xlabel="",
            ylabel="",
            stacked=True,
            cmap=["lightgray", "green"],
            legend=False,
        ).opts(**plot_opts, legend_position="top_right")
        cat_plots.append(plot)
    return hv.Layout(cat_plots).cols(3)


def plot_num_active_violins(df, list_of_num_cols, target_col="adopted_user"):
    """Create 2 violin plot for each numerical column with the target column."""

    plot_opts = dict(height=300, width=400, active_tools=["box_zoom"])
    num_plots = []
    for col in list_of_num_cols:
        plot = df.hvplot.violin(
            y=col,
            by=target_col,
            c=target_col,
            ylabel="",
            title=f"{col}".replace("_", " ").title(),
            cmap=["lightgray", "green"],
        ).opts(**plot_opts)
        num_plots.append(plot)
    return hv.Layout(num_plots).cols(3)


def plot_num_active_hist(df, list_of_num_cols, target_col="adopted_user"):
    """Creates a histogram plot of each numerical column with target column
    separated by color."""

    plot_opts = dict(height=300, width=400, active_tools=["box_zoom"])
    hist_plots = [
        df[[num_col] + [target_col]]
        .hvplot.hist(
            title=f"{num_col} Distribution",
            bins=12,
            xlabel="",
            by=target_col,
            alpha=0.6,
            muted_alpha=0.02,
            color=["silver", "green"],
        )
        .opts(**plot_opts)
        for num_col in list_of_num_cols
    ]
    return hv.Layout(hist_plots)


def plot_cat_bars(df, list_of_cat_cols):
    """Creates a bar plot for each categorical column."""

    plot_opts = dict(height=300, width=400, active_tools=["box_zoom"])
    cat_plots = [
        df[col]
        .astype("string")
        .value_counts()
        .sort_index()
        .hvplot.barh(
            title=f"{col}".replace("_", " ").title() if "_" in col else col.title(),
            xlabel="",
            ylabel="",
            color="silver",
        )
        .opts(**plot_opts)
        for col in list_of_cat_cols
    ]
    return hv.Layout(cat_plots).cols(3)

In [252]:
cat_cols = [
    "domain",
    "creation_source",
    "was_invited",
    "creation_month",
    "sent_referrals",
    "invited_self",
    "opted_in_to_mailing_list",
    "enabled_for_marketing_drip",
]

all_users_df.dtypes
plot_cat_active_stacked_bars(all_users_df, cat_cols)

:Layout
   .Bars.I    :Bars   [domain,adopted_user]   (value)
   .Bars.II   :Bars   [creation_source,adopted_user]   (value)
   .Bars.III  :Bars   [was_invited,adopted_user]   (value)
   .Bars.IV   :Bars   [creation_month,adopted_user]   (value)
   .Bars.V    :Bars   [sent_referrals,adopted_user]   (value)
   .Bars.VI   :Bars   [invited_self,adopted_user]   (value)
   .Bars.VII  :Bars   [opted_in_to_mailing_list,adopted_user]   (value)
   .Bars.VIII :Bars   [enabled_for_marketing_drip,adopted_user]   (value)

In [253]:
plot_cat_bars(all_users_df, cat_cols)

:Layout
   .Bars.Count.I    :Bars   [domain]   (count)
   .Bars.Count.II   :Bars   [creation_source]   (count)
   .Bars.Count.III  :Bars   [was_invited]   (count)
   .Bars.Count.IV   :Bars   [creation_month]   (count)
   .Bars.Count.V    :Bars   [sent_referrals]   (count)
   .Bars.Count.VI   :Bars   [invited_self]   (count)
   .Bars.Count.VII  :Bars   [opted_in_to_mailing_list]   (count)
   .Bars.Count.VIII :Bars   [enabled_for_marketing_drip]   (count)

In [254]:
plot_num_active_violins(
    all_users_df,
    [
        "frequency",
        "account_age_last_login",
        "num_referrals",
        "account_age",
        "days_since_last_login",
        "time_bet_logins",
    ],
)

:Layout
   .Violin.I   :Violin   [adopted_user]   (frequency)
   .Violin.II  :Violin   [adopted_user]   (account_age_last_login)
   .Violin.III :Violin   [adopted_user]   (num_referrals)
   .Violin.IV  :Violin   [adopted_user]   (account_age)
   .Violin.V   :Violin   [adopted_user]   (days_since_last_login)
   .Violin.VI  :Violin   [adopted_user]   (time_bet_logins)

In [255]:
# find the correct transform for the right skewed columns
plot_num_active_hist(
    all_users_df,
    ["frequency", "account_age_last_login", "num_referrals", "account_age"],
).cols(3)

stripped_outliers = []
for col in ["frequency", "account_age_last_login", "num_referrals", "account_age"]:
    col_stripped = remove_outer_percentile_outliers(all_users_df, col)

    stripped_outliers.append(plot_num_active_hist(col_stripped, [col]))

hv.Layout(stripped_outliers).cols(3)

:Layout
   .NdOverlay.I   :NdOverlay   [adopted_user]
      :Histogram   [frequency]   (frequency_count)
   .NdOverlay.II  :NdOverlay   [adopted_user]
      :Histogram   [account_age_last_login]   (account_age_last_login_count)
   .NdOverlay.III :NdOverlay   [adopted_user]
      :Histogram   [num_referrals]   (num_referrals_count)
   .NdOverlay.IV  :NdOverlay   [adopted_user]
      :Histogram   [account_age]   (account_age_count)

In [256]:
adopted_col_features = [
    "time_to_become_adopted",
    "avg_time_bet_logins_at_adopt",
    "login_count_at_adopt",
    "account_age_at_adopt",
    "days_since_last_login",
    "time_bet_logins",
    "login_count_at_adopt",
]
stripped_outliers = []
for col in adopted_col_features:
    col_stripped = remove_outer_percentile_outliers(all_users_df, col)

    stripped_outliers.append(plot_num_active_hist(
        col_stripped,
        [col],
    ))

hv.Layout(stripped_outliers).cols(3)

:Layout
   .NdOverlay.I   :NdOverlay   [adopted_user]
      :Histogram   [time_to_become_adopted]   (time_to_become_adopted_count)
   .NdOverlay.II  :NdOverlay   [adopted_user]
      :Histogram   [avg_time_bet_logins_at_adopt]   (avg_time_bet_logins_at_adopt_count)
   .NdOverlay.III :NdOverlay   [adopted_user]
      :Histogram   [login_count_at_adopt]   (login_count_at_adopt_count)
   .NdOverlay.IV  :NdOverlay   [adopted_user]
      :Histogram   [account_age_at_adopt]   (account_age_at_adopt_count)
   .NdOverlay.V   :NdOverlay   [adopted_user]
      :Histogram   [days_since_last_login]   (days_since_last_login_count)
   .NdOverlay.VI  :NdOverlay   [adopted_user]
      :Histogram   [time_bet_logins]   (time_bet_logins_count)
   .NdOverlay.VII :NdOverlay   [adopted_user]
      :Histogram   [login_count_at_adopt]   (login_count_at_adopt_count)

In [257]:
all_users_df.dtypes.sort_index()
all_users_df.select_dtypes(np.number).columns
# get the org_id which has the most ORG_INVITE creation_source
org_invite = all_users_df[all_users_df["creation_source"] == "ORG_INVITE"]
org_invite.sort_values(by="sent_referrals", ascending=False)

,user_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,recency,recency_cluster,frequency,frequency_cluster,num_referrals,referral_cluster,date,adopted_user,time_to_become_adopted,avg_time_bet_logins_at_adopt,login_count_at_adopt,account_age_at_adopt,days_since_last_login,start_date,account_age,account_age_last_login,time_bet_logins,creation_month,domain,was_invited,sent_referrals,invited_self
3920,05370,2013-04-05 19:00:43,Alves Gabrielly,GabriellyBarbosaAlves@gmail.com,ORG_INVITE,2013-04-06 19:00:43,1,1,358,04771,425.0,1,1,0,1,1,NaN,0,NaN,NaN,NaN,NaN,425,2013-04-05,426,1,426.0,4,gmail,Invited,sent_referrals,0
7378,10042,2013-05-24 03:48:01,Almeida Sophia,SophiaCardosoAlmeida@yahoo.com,ORG_INVITE,2013-05-24 03:48:01,0,0,374,10221,378.0,1,1,0,1,1,NaN,0,NaN,NaN,NaN,NaN,378,2013-05-24,378,0,378.0,5,yahoo,Invited,sent_referrals,0
3805,05202,2012-07-05 09:31:37,Rasmussen Mohammad,MohammadMRasmussen@hotmail.com,ORG_INVITE,2012-07-08 09:31:37,0,0,53,00093,698.0,0,1,0,2,1,NaN,0,NaN,NaN,NaN,NaN,698,2012-07-05,701,3,701.0,7,hotmail,Invited,sent_referrals,0
1641,02263,2012-09-04 13:38:37,Eisenhauer Melanie,avdcbbaz@wlpro.com,ORG_INVITE,2012-09-04 13:38:37,0,0,406,09785,640.0,0,1,0,1,1,NaN,0,NaN,NaN,NaN,NaN,640,2012-09-04,640,0,640.0,9,other,Invited,sent_referrals,0
3803,05200,2012-11-14 15:31:49,Rhodes Victoria,VictoriaRhodes@cuvox.de,ORG_INVITE,2014-05-26 15:31:49,1,1,97,10649,10.0,2,258,1,1,1,2013-01-15,1,61.0,6.1,10.0,61.0,10,2012-11-14,568,558,2.2,11,cuvox,Invited,sent_referrals,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3259,04440,2014-04-04 04:05:56,Akhtar Emily,EmilyAkhtar@jourrapide.com,ORG_INVITE,2014-04-06 04:05:56,0,0,128,04612,61.0,2,1,0,0,0,NaN,0,NaN,NaN,NaN,NaN,61,2014-04-04,63,2,63.0,4,jourrapide,Invited,no_referrals_sent,0
3261,04442,2014-03-27 08:46:14,Ribeiro Marisa,MarisaCavalcantiRibeiro@jourrapide.com,ORG_INVITE,2014-03-27 08:46:14,1,0,101,07630,71.0,2,1,0,0,0,NaN,0,NaN,NaN,NaN,NaN,71,2014-03-27,71,0,71.0,3,jourrapide,Invited,no_referrals_sent,0
3268,04450,2014-03-08 19:22:51,Bradley Courtney,CourtneyDBradley@hotmail.com,ORG_INVITE,2014-03-09 19:22:51,0,0,140,11040,88.0,2,1,0,0,0,NaN,0,NaN,NaN,NaN,NaN,88,2014-03-08,89,1,89.0,3,hotmail,Invited,no_referrals_sent,0
3271,04453,2014-04-29 20:04:45,Bang Christine,pbnkmgsx@kfumy.com,ORG_INVITE,2014-05-07 20:04:45,0,0,19,06896,29.0,2,2,0,0,0,NaN,0,NaN,NaN,NaN,NaN,29,2014-04-29,37,8,18.5,4,other,Invited,no_referrals_sent,0


In [258]:
# Get the org_id which has the highest portion of adopted users compared to its total users
# bin them into 5 groups
all_users_df["org_id_adopted_ratio"] = all_users_df.groupby("org_id")[
    "adopted_user"
].transform("mean")
all_users_df["org_id_count"] = all_users_df["org_id"].map(
    all_users_df["org_id"].value_counts()
)

# create an organiztion dataframe with the org_id as the index and put the descriptions in the columns
org_df = all_users_df.groupby("org_id")["org_id_adopted_ratio"].mean().reset_index()
org_df["org_id_count"] = org_df["org_id"].map(all_users_df["org_id"].value_counts())
# add a column with thhe day the organization first logged in
org_df["org_first_login"] = all_users_df.groupby("org_id")["creation_time"].min()
# Add a column with the day the organization last logged in
org_df["org_last_login"] = all_users_df.groupby("org_id")[
    "last_session_creation_time"
].max()
# get the account age for the organization
org_df["org_account_age"] = (
    org_df["org_last_login"] - org_df["org_first_login"]
).dt.days
# get the number of logins for the organization
org_df["number_of_logins"] = all_users_df.groupby(["org_id"])["frequency"].sum()
# get the number of adopted users for the organization
org_df["number_of_adopted_users"] = all_users_df.groupby(["org_id"])[
    "adopted_user"
].sum()

all_users_df.hvplot.scatter(
    y="account_age",
    x="frequency",
    c="adopted_user",
    size=2,
    alpha=0.7,
    height=500,
    width=800,
    hover_cols=["user_id"],
    grid=True,
).opts(active_tools=["box_zoom"], color_levels=5)

:Scatter   [frequency]   (account_age,adopted_user,user_id)

In [193]:
# check correlations of the numerical columns with logins
all_users_df.corr(numeric_only=True)[
    "adopted_user"].sort_values(ascending=False)

adopted_user                    1.000000
account_age_last_login          0.777844
frequency                       0.625010
frequency_cluster               0.598727
recency_cluster                 0.338938
time_bet_logins                 0.253677
org_id_adopted_ratio            0.247827
account_age                     0.108571
org_id                          0.077748
referral_cluster                0.055126
num_referrals                   0.049752
creation_month                  0.047582
invited_self                    0.032707
opted_in_to_mailing_list        0.007485
enabled_for_marketing_drip      0.004290
org_id_count                   -0.099119
days_since_last_login          -0.392351
recency                        -0.392351
time_to_become_adopted               NaN
avg_time_bet_logins_at_adopt         NaN
login_count_at_adopt                 NaN
account_age_at_adopt                 NaN
Name: adopted_user, dtype: float64

In [259]:
# create a copy of the dataframe of the user engagement
new_users_engagement_df = users_engagement_df.copy()
# convert the user_id to string and add leading zeros
new_users_engagement_df["user_id"] = (
    new_users_engagement_df["user_id"].astype("string").str.zfill(5)
)
# merge the user engagement with the all users dataframe
org_time_df = new_users_engagement_df.merge(all_users_df[["org_id", "user_id"]]).drop(
    ["user_id", "date"], axis=1
)
# convert the time_stamp to datetime
org_time_df["time_stamp"] = pd.to_datetime(org_time_df["time_stamp"])

# resample timestamp to hourly intervals and count the number of logins
# org_hourly = org_time_df.set_index("time_stamp").groupby("org_id").resample("H").count()

# Extract hour of day from timestamp
org_time_df["hour_of_day"] = org_time_df["time_stamp"].dt.hour

# Group by org_id and hour of day, and count the number of logins
org_hourly = org_time_df.groupby(["org_id", "hour_of_day"]).count()


# unsttack the org_id level of the MultiIndex
org_df_unstacked = org_hourly.unstack(level=0).fillna(0)
print(f"Shape of the unstacked DataFrame: {org_df_unstacked.shape}")
org_df_unstacked.columns = org_df_unstacked.columns.droplevel()
org_hourly_df = org_df_unstacked.T.reset_index()
org_hourly_df = org_hourly_df.set_index("org_id").rename_axis(None, axis=1)
org_hourly_df

# Standardize the data
scaler = StandardScaler()
org_df_unstacked_scaled = scaler.fit_transform(org_hourly_df)

# Define the number of clusters you want to find
n_clusters = 26

# Apply K-means clustering
kmeans = KMeans(n_clusters=n_clusters, random_state=628).fit(org_df_unstacked_scaled)


# add the cluster labels to the org_daily_df
org_hourly_df = org_hourly_df.reset_index()
org_hourly_df["org_group"] = kmeans.labels_
org_hourly_df[["org_id", "org_group"]]
# org_daily_df

Shape of the unstacked DataFrame: (24, 417)


,org_id,org_group
0,0,22
1,1,10
2,2,3
3,3,12
4,4,10
...,...,...
412,412,25
413,413,1
414,414,25
415,415,6


In [260]:
# Merge on org_id to put the org_group in the all_users_df

org_grouped_all_users = all_users_df.merge(
    org_hourly_df[["org_id", "org_group"]],
    how="left",
)

plot_cat_bars(org_grouped_all_users, ["org_group"]) + plot_cat_active_stacked_bars(
    org_grouped_all_users, ["org_group"]
)

:Layout
   .Bars.Count :Bars   [org_group]   (count)
   .Bars.I     :Bars   [org_group,adopted_user]   (value)

In [359]:
# show the users who are not adopted but have a high number of logins
all_users_df[(all_users_df["logins"] > 3)
             & (all_users_df["adopted_user"] == 0)].sort_values(
                 by=["logins"], ascending=False)

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,user_id,logins,date,adopted_user,time_to_become_adopted,avg_time_bet_logins_at_adopt,login_count_at_adopt,account_age_at_adopt,days_since_last_login,start_date,account_age,account_age_last_login,login_quietness,creation_month,domain,num_referred,was_invited,sent_referrals,invited_self,org_id_adopted_ratio,org_id_count
8329,11321,2013-01-30 18:48:54,Butler Aimee,AimeeButler@yahoo.com,ORG_INVITE,2013-07-05 18:48:54,0,0,158,03515,11321,15,NaN,0,NaN,NaN,NaN,NaN,335,2013-01-30,491,156,10.4,1,yahoo,0,Invited,no_referrals_sent,0,0.058824,17
4666,6371,2012-11-21 11:01:28,Shepherd Lenard,LenardMShepherd@gmail.com,SIGNUP_GOOGLE_AUTH,2013-05-06 11:01:28,0,0,160,Not Invited,06371,13,NaN,0,NaN,NaN,NaN,NaN,396,2012-11-21,562,166,12.8,11,gmail,0,Not Invited,no_referrals_sent,0,0.352941,17
1546,2134,2012-12-08 03:58:08,Schröder Andrea,AndreaSchroder@yahoo.com,ORG_INVITE,2013-04-26 03:58:08,0,0,129,02042,02134,13,NaN,0,NaN,NaN,NaN,NaN,406,2012-12-08,545,139,10.7,12,yahoo,0,Invited,no_referrals_sent,0,0.238095,21
8056,10949,2013-03-17 11:26:42,Crowder Mason,MasonCrowder@gmail.com,ORG_INVITE,2013-07-30 11:26:42,0,0,106,08623,10949,12,NaN,0,NaN,NaN,NaN,NaN,311,2013-03-17,446,135,11.2,3,gmail,0,Invited,no_referrals_sent,0,0.208333,24
300,398,2013-06-10 17:48:12,Gomes Igor,IgorRochaGomes@gmail.com,ORG_INVITE,2013-09-01 17:48:12,0,0,105,01758,00398,12,NaN,0,NaN,NaN,NaN,NaN,277,2013-06-10,360,83,6.9,6,gmail,0,Invited,no_referrals_sent,0,0.150000,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2574,3521,2014-05-07 10:47:50,Sadler Abbey,AbbeySadler@jourrapide.com,GUEST_INVITE,2014-06-01 10:47:50,0,0,244,04215,03521,4,NaN,0,NaN,NaN,NaN,NaN,5,2014-05-07,30,25,6.2,5,jourrapide,0,Invited,no_referrals_sent,0,0.090909,11
2535,3470,2013-01-05 08:08:30,Humphries Evan,EvanHumphries@gmail.com,GUEST_INVITE,2013-03-22 08:08:30,0,0,327,01775,03470,4,NaN,0,NaN,NaN,NaN,NaN,441,2013-01-05,517,76,19.0,1,gmail,2,Invited,sent_referrals,0,0.071429,14
5960,8173,2012-10-14 12:19:57,Code Emma,EmmaCode@gustr.com,SIGNUP,2012-11-18 12:19:57,0,0,163,Not Invited,08173,4,NaN,0,NaN,NaN,NaN,NaN,565,2012-10-14,600,35,8.8,10,gustr,0,Not Invited,no_referrals_sent,0,0.333333,21
6027,8254,2014-04-22 07:11:26,Fernandes Luis,LuisDiasFernandes@yahoo.com,ORG_INVITE,2014-05-25 07:11:26,0,0,250,07572,08254,4,NaN,0,NaN,NaN,NaN,NaN,12,2014-04-22,45,33,8.2,4,yahoo,0,Invited,no_referrals_sent,0,0.250000,12


In [261]:
new_df = all_users_df.copy()

new_df["is_referrer"] = new_df["sent_referrals"].apply(
    lambda x: 1 if x in ("sent_referrals") else 0
)

# Test the Active Level column
new_df["is_home_project"] = new_df["creation_source"].apply(
    lambda x: 1 if x in ("GUEST_INVITE", "PERSONAL_PROJECTS") else 0
)

new_df["active_level"] = (
    new_df["is_referrer"] + new_df["is_home_project"] + new_df["invited_self"]
)
hv.Layout(
    [
        plot_cat_bars(new_df, ["active_level"]),
        plot_cat_active_stacked_bars(new_df, ["active_level"]),
    ]
)

:Layout
   .Bars.Count :Bars   [active_level]   (count)
   .Bars.I     :Bars   [active_level,adopted_user]   (value)

In [262]:
# combine the domains to 'all_others' except for hotmail and yahoo
new_df["new_domain"] = new_df["domain"].apply(
    lambda x: "all_others" if x not in ("hotmail", "yahoo") else x
)
(
    plot_cat_active_stacked_bars(new_df, ["new_domain"])
    + plot_cat_bars(new_df, ["new_domain"])
)

:Layout
   .Bars.I     :Bars   [new_domain,adopted_user]   (value)
   .Bars.Count :Bars   [new_domain]   (count)

In [263]:
# create feature as the sum of the 2 spam columns
new_df["spam_prod"] = (
    new_df["opted_in_to_mailing_list"] * new_df["enabled_for_marketing_drip"]
)
new_df["spam_sum"] = (
    new_df["opted_in_to_mailing_list"] + new_df["enabled_for_marketing_drip"]
)
hv.Layout(
    [
        plot_cat_active_stacked_bars(new_df, ["spam_prod"]),
        plot_cat_active_stacked_bars(new_df, ["spam_sum"]),
    ]
)

:Layout
   .Bars.I  :Bars   [spam_prod,adopted_user]   (value)
   .Bars.II :Bars   [spam_sum,adopted_user]   (value)

No difference with the combining the spam features. ow we try combining the `new_domain` wiht the spam features.

In [264]:
# multiply the spam prod and the domain columns
new_df["spam_prod"] = new_df["spam_prod"].astype("string")
new_df["spam_domain"] = new_df["new_domain"] + "_" + new_df["spam_prod"]
(
    plot_cat_active_stacked_bars(new_df, ["spam_domain"])
    + plot_cat_bars(new_df, ["spam_domain"])
)

:Layout
   .Bars.I     :Bars   [spam_domain,adopted_user]   (value)
   .Bars.Count :Bars   [spam_domain]   (count)

- Only the yahoo domain seems to respond positively to the email marketing. 
- Hotmail domains were negatively affected by the email marketing. 
- All the others did not have much of a difference. (< 1% difference)


In [265]:
new_df["spam_source"] = (
    new_df["creation_source"] + "_" + new_df["spam_prod"].astype("string")
)

(
    plot_cat_active_stacked_bars(new_df, ["spam_source"])
    + plot_cat_bars(new_df, ["spam_source"])
)

:Layout
   .Bars.I     :Bars   [spam_source,adopted_user]   (value)
   .Bars.Count :Bars   [spam_source]   (count)

- Those who `creation_source` is `PERSONAL_PROJECTS` have a positive response to the email marketing. 
- All others are indifferent ( < 2% difference).


In [266]:
all_users_df.dtypes.sort_index()

account_age                              int64
account_age_at_adopt                   float64
account_age_last_login                   int64
adopted_user                             int32
avg_time_bet_logins_at_adopt           float64
creation_month                           int32
creation_source                         object
creation_time                   datetime64[ns]
date                                    object
days_since_last_login                    int64
domain                                  object
email                                   object
enabled_for_marketing_drip               int64
frequency                                int32
frequency_cluster                        int64
invited_by_user_id              string[python]
invited_self                             int32
last_session_creation_time      datetime64[ns]
login_count_at_adopt                   float64
name                                    object
num_referrals                            int32
opted_in_to_m

In [267]:
# spam_domain feature
all_users_df["new_domain"] = all_users_df["domain"].apply(
    lambda x: "all_others" if x not in ("hotmail", "yahoo") else x
)
all_users_df["spam_prod"] = (
    all_users_df["opted_in_to_mailing_list"]
    * all_users_df["enabled_for_marketing_drip"]
)
all_users_df["spam_domain"] = (
    all_users_df["new_domain"] + "_" +
    all_users_df["spam_prod"].astype("string")
)
# active_level feature
all_users_df["is_referrer"] = all_users_df["sent_referrals"].apply(
    lambda x: 1 if x in ("sent_referrals") else 0
)
all_users_df["is_home_project"] = all_users_df["creation_source"].apply(
    lambda x: 1 if x in ("GUEST_INVITE", "PERSONAL_PROJECTS") else 0
)
all_users_df["active_level"] = (
    all_users_df["is_referrer"]
    + all_users_df["is_home_project"]
    + all_users_df["invited_self"]
)

In [268]:
categorical_features = [
    "active_level",
    "spam_domain",
    "creation_month",
    "sent_referrals",
    # 'org_id',
    "num_referrals",
]
numerical_features = ["days_since_last_login"]
# binning_features = ["num_referrals"]

X = all_users_df[
    categorical_features
    + numerical_features
    # + binning_features
]
y = all_users_df["adopted_user"]

In [269]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=628, stratify=y
)

In [383]:
from category_encoders import TargetEncoder

In [270]:
# create a pipeline for numerical columns
numerical_pipeline = Pipeline(
    steps=[
        ("scaler", RobustScaler()),
    ]
)

# create a pipeline for categorical columns
categorical_pipeline = Pipeline(
    steps=[
        # ('target', TargetEncoder()),
        ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False)),
    ]
)
# create a pipeline for binning features
# binning_pipeline = Pipeline(steps=[
#     (
#         "discretizer",
#         KBinsDiscretizer(
#             n_bins=4, encode="ordinal", strategy="uniform", subsample=10000),
#     ),
#     ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False)),
# ])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numerical_pipeline, numerical_features),
        ("cat", categorical_pipeline, categorical_features),
        # ("bin", binning_pipeline, binning_features),
    ],
    remainder="drop",
)

In [271]:
# Models dict
models = {}
models["LogisticRegression"] = (
    LogisticRegression(
        random_state=628,
        n_jobs=-1,
        max_iter=1000,
    ),
    {
        "classifier__C": np.logspace(-3, 3, 7),
        "classifier__class_weight": ["balanced", None],
        "classifier__solver":
        ["newton-cg", "lbfgs", "liblinear", "sag", "saga"],
    },
)

models["LightGBM"] = (
    LGBMClassifier(
        random_state=628,
        n_jobs=-1,
        is_unbalance=True,
        num_leaves=31,
        boosting_type="gbdt",
        verbose=-1,
    ),
    {
        "classifier__learning_rate": stats.uniform(0.01, 0.5),
        "classifier__reg_alpha": stats.uniform(0.0, 0.05),
        "classifier__reg_lambda": stats.uniform(0.0, 0.05),
        "classifier__min_child_samples": stats.randint(100, 500),
        "classifier__min_data_in_leaf": stats.randint(20, 100),
    },
)

models["XGBoost"] = (
    XGBClassifier(
        random_state=628,
        scale_pos_weight=sum(y_train == 0) / sum(y_train == 1),
    ),
    {
        "classifier__n_estimators": stats.randint(100, 1000),
        "classifier__learning_rate": [0.001, 0.01, 0.1, 0.2, 0.4, 0.5],
        "classifier__max_depth": stats.randint(3, 10),
        "classifier__subsample": stats.uniform(0.6, 0.4),
        "classifier__colsample_bytree": stats.uniform(0.6, 0.4),
        "classifier__colsample_bylevel": stats.uniform(0.6, 0.4),
        "classifier__min_child_weight": stats.randint(1, 200),
    },
)

In [272]:
# Create an empty dictionary to store the best models
best_models = {}

# Loop through each model and perform random search
for model_name, (model, param_grid) in tqdm(models.items(), desc="Model Tuning"):
    # Create a pipeline for the model
    model_pipeline = Pipeline(
        steps=[
            ("preprocessor", preprocessor),
            ("classifier", model),
        ]
    )

    # Perform RandomizedSearchCV
    random_search = RandomizedSearchCV(
        model_pipeline,
        param_grid,
        n_iter=10,
        cv=5,
        random_state=628,
        scoring="recall",
        n_jobs=-1,
    )

    # Fit the model
    random_search.fit(X_train, y_train)

    # Store the best model
    best_models[model_name] = random_search.best_estimator_

    # Print the best parameters and ROC AUC score for each model
    print(f"\nBest Parameters for {model_name}: {random_search.best_params_}")
    # Print the mean cross-validation score for the best parameters
    best_index = random_search.best_index_
    mean_cv_score = random_search.cv_results_["mean_test_score"][best_index]
    print(f"{model_name} - Mean CV Score for Best Parameters: {mean_cv_score:.3f}")
    y_pred_proba_tuned = random_search.best_estimator_.predict_proba(X_test)[
        :, 1]
    y_pred = best_models[model_name].predict(X_test)
    roc_auc_tuned = roc_auc_score(y_test, y_pred_proba_tuned)
    recall_score_tuned = recall_score(y_test, y_pred)

    print(f"{model_name} - Tuned Model ROC AUC Score: {roc_auc_tuned:.3f}")
    print(f"{model_name} - Tuned Model Recall Score:{recall_score_tuned:.3f}")

Model Tuning:   0%|          | 0/3 [00:00<?, ?it/s]

b:\project\ultimate_project_3\venv\lib\site-packages\sklearn\linear_model\_logistic.py:1216: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  warnings.warn(



Best Parameters for LogisticRegression: {'classifier__solver': 'liblinear', 'classifier__class_weight': 'balanced', 'classifier__C': 0.01}
LogisticRegression - Mean CV Score for Best Parameters: 0.826
LogisticRegression - Tuned Model ROC AUC Score: 0.879
LogisticRegression - Tuned Model Recall Score:0.807

Best Parameters for LightGBM: {'classifier__learning_rate': 0.07827724005998087, 'classifier__min_child_samples': 366, 'classifier__min_data_in_leaf': 83, 'classifier__reg_alpha': 0.017503009363665636, 'classifier__reg_lambda': 0.004726102044580444}
LightGBM - Mean CV Score for Best Parameters: 0.898
LightGBM - Tuned Model ROC AUC Score: 0.974
LightGBM - Tuned Model Recall Score:0.882

Best Parameters for XGBoost: {'classifier__colsample_bylevel': 0.685450405251666, 'classifier__colsample_bytree': 0.7402143706669888, 'classifier__learning_rate': 0.2, 'classifier__max_depth': 4, 'classifier__min_child_weight': 29, 'classifier__n_estimators': 897, 'classifier__subsample': 0.8476701101

In [273]:
feature_names = (
    best_models["LogisticRegression"]
    .named_steps["preprocessor"]
    .get_feature_names_out()
)

lr_importance = best_models["LogisticRegression"].named_steps["classifier"].coef_[0]

xgb_feature_importance = (
    best_models["XGBoost"].named_steps["classifier"].feature_importances_
)

lgbm_feature_importance = (
    best_models["LightGBM"].named_steps["classifier"].feature_importances_
)

In [274]:
# Create a DataFrame for easy visualization
feature_importances_df = pd.DataFrame(
    {
        "Logistic Regression": lr_importance,
        "XGBoost": xgb_feature_importance,
        "Light GBM": lgbm_feature_importance,
    },
    index=feature_names,
)


def highlight_topn(s):
    is_top3 = s.isin(s.nlargest(4))
    return ["background-color: salmon" if v else "" for v in is_top3]


# Sort features by importance
feature_importances_df.sort_values("Logistic Regression", ascending=False, inplace=True)
# highlight the 3 highest values in each column
feature_importances_df.style.apply(highlight_topn, axis=0)
# feature_importances_df.round(4)

,Logistic Regression,XGBoost,Light GBM
cat__sent_referrals_sent_referrals,0.444425,0.041074,1
cat__creation_month_6,0.412614,0.052005,66
cat__active_level_2,0.256732,0.017393,4
cat__creation_month_7,0.243438,0.033518,53
cat__creation_month_8,0.229871,0.031349,49
cat__spam_domain_hotmail_0,0.170426,0.016976,26
cat__creation_month_9,0.168116,0.041242,62
cat__creation_month_11,0.155816,0.034805,68
cat__creation_month_10,0.151397,0.029442,69
cat__num_referrals_4,0.127478,0.000000,5


In [275]:
# Plot feature importances
active_opts = dict(
    active_tools=["box_zoom"],
)
hv.Layout(
    [
        feature_importances_df[col]
        .hvplot.barh(
            y=col,
            xlabel="",
            #  title with model name
            title=f"Feature Importance {col}",
            height=600,
        )
        .opts(**active_opts)
        for col in feature_importances_df.columns
    ],
).cols(2)

:Layout
   .Bars.Logistic_Regression :Bars   [index]   (Logistic Regression)
   .Bars.XGBoost             :Bars   [index]   (XGBoost)
   .Bars.Light_GBM           :Bars   [index]   (Light GBM)

In [282]:
def get_roc_confusion_matrix(model, X_test, y_test):
    """Get the ROC AUC score and the confusion matrix for the model."""
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    # y_pred = model.predict(X_test)

    # Get the ROC AUC score
    roc_auc = roc_auc_score(y_test, y_pred_proba)

    # Get the confusion matrix
    fpr, tpr, thresh = roc_curve(y_test, y_pred_proba)
    # Calculate the optimal threshold
    J = tpr - fpr
    optimal_idx = np.argmax(J)
    optimal_threshold = thresh[optimal_idx]

    # Use the optimal threshold to convert probabilities into class predictions
    y_pred = (y_pred_proba >= optimal_threshold).astype(int)

    roc_curve_plot = hv.Curve((fpr, tpr)).opts(
        title=f"ROC Curve (AUC: {roc_auc:.2f} || opt thresh:{optimal_threshold:.2f})",  # noqa: E501
        xlabel="False Positive Rate",
        ylabel="True Positive Rate",
        line_width=2,
        height=400,
        width=400,
        tools=["hover"],
        active_tools=["box_zoom"],
    )
    conf_matrix = confusion_matrix(y_test, y_pred)
    conf_matrix = pd.DataFrame(
        conf_matrix,
        index=["Actual 0", "Actual 1"],
        columns=["Predicted 0", "Predicted 1"],
    )
    print(classification_report(y_test, y_pred))
    conf_heatmap = conf_matrix.hvplot.heatmap(
        height=400, width=400, colorbar=False, title="Confusion Matrix"
    ).opts(
        **active_opts,
        invert_yaxis=True,
        color_levels=5,
        line_color="white",
        line_width=2,
    )
    # Create a DataFrame for labels
    labels_df = pd.DataFrame(conf_matrix.stack(), columns=["value"]).reset_index()
    labels_df.columns = ["y", "x", "value"]

    # Create labels
    labels = hv.Labels(labels_df, ["x", "y"], "value")

    return hv.Layout(
        [
            (conf_heatmap * labels.opts(text_color="gray", text_font_size="14pt")),
            roc_curve_plot,
        ]
    ).cols(2)

Logistic Regression

In [283]:
lr_model = best_models["LogisticRegression"]
get_roc_confusion_matrix(lr_model, X_test, y_test)

              precision    recall  f1-score   support

           0       0.94      0.97      0.95      1434
           1       0.84      0.71      0.77       331

    accuracy                           0.92      1765
   macro avg       0.89      0.84      0.86      1765
weighted avg       0.92      0.92      0.92      1765



:Layout
   .Overlay.I :Overlay
      .HeatMap.I :HeatMap   [columns,index]   (value)
      .Labels.I  :Labels   [x,y]   (value)
   .Curve.I   :Curve   [x]   (y)

XGBoost

In [284]:
xgboost_model = best_models["XGBoost"]
get_roc_confusion_matrix(xgboost_model, X_test, y_test)

              precision    recall  f1-score   support

           0       0.97      0.95      0.96      1434
           1       0.80      0.87      0.83       331

    accuracy                           0.93      1765
   macro avg       0.88      0.91      0.89      1765
weighted avg       0.94      0.93      0.93      1765



:Layout
   .Overlay.I :Overlay
      .HeatMap.I :HeatMap   [columns,index]   (value)
      .Labels.I  :Labels   [x,y]   (value)
   .Curve.I   :Curve   [x]   (y)

LightGBM

In [285]:
lgbm_model = best_models["LightGBM"]
get_roc_confusion_matrix(lgbm_model, X_test, y_test)

              precision    recall  f1-score   support

           0       0.98      0.92      0.95      1434
           1       0.74      0.92      0.82       331

    accuracy                           0.92      1765
   macro avg       0.86      0.92      0.88      1765
weighted avg       0.93      0.92      0.93      1765



:Layout
   .Overlay.I :Overlay
      .HeatMap.I :HeatMap   [columns,index]   (value)
      .Labels.I  :Labels   [x,y]   (value)
   .Curve.I   :Curve   [x]   (y)